In [1]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F


class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x
    
    
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

    
class ChannelGate(nn.Module):
    '''Generate 2 different(avg, pool) spatial context descriptors to refine input feature'''
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels

        # Shared MLP
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),  # reduction
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)  # restoration
            )
        
        self.pool_types = pool_types

    def forward(self, x):
        '''x: Input feature  (N, C, h, w)
           kernel_size of pooling operation = (h, w) -> squeeze the spatial dimension'''
        channel_att_sum = None  # It should be MLP(AvgPool(x)) + MLP(MaxPool(x))
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))  # (N, C, 1, 1)
                channel_att_raw = self.mlp(avg_pool)  # (N, C)

            elif pool_type=='max':
                max_pool = F.max_pool2d(x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))  # (N, C, 1, 1)
                channel_att_raw = self.mlp(max_pool)  # (N, C)

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw

            else:
                channel_att_sum = channel_att_sum + channel_att_raw  # (N, C) - Channel Attention Map
        
        # Sigmoid & Broad-casting (N, C) -> (N, C, 1) -> (N, C, 1, 1) -> (N, C, h, w)
        scale = F.sigmoid(channel_att_sum).unsqueeze(2).unsqueeze(3).expand_as(x)  
        
        # Feature Refinement 
        return x * scale  # (N, C, h, w)
    
    
class ChannelPool(nn.Module):
    '''Apply max pooling & avg pooling along the channel axis and concatenate them
       to generate an efficient feature descriptor'''
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), torch.mean(x, 1).unsqueeze(1)), dim=1)

    
class SpatialGate(nn.Module):
    '''Produce 2D spatial attention map to refine channel-refined feature (sequential)'''
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)

    def forward(self, x):
        '''x: channel-refined feature (sequential)'''
        x_compress = self.compress(x)  # (N, 2, h, w)
        x_out = self.spatial(x_compress)  # (N, 1, h, w) - Spatial Attention Map
        scale = F.sigmoid(x_out)  # broadcasting

        return x * scale
    

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial = no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()

    def forward(self, x):
        '''x: Input feature'''
        x_out = self.ChannelGate(x)  # Channel-refinement
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)  # Spatial-refinement

        return x_out  # Refined feature

In [2]:
import torch
from torch import nn
#from cbam import CBAM

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

        self.cbam = CBAM(out_ch)

    def forward(self, input):
        output = self.conv(input)
        output = self.cbam(output)
        
        return output

In [24]:
class Unet_CBAM(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(Unet_CBAM, self).__init__()
        
        # Down-sampling
        self.conv1 = DoubleConv(in_ch, 64)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        self.conv5 = DoubleConv(512, 1024)
        
        # Up-sampling
        self.up6 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv6 = DoubleConv(1024, 512)

        self.up7 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv7 = DoubleConv(512, 256)

        self.up8 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv8 = DoubleConv(256, 128)

        self.up9 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv9 = DoubleConv(128, 64)

        self.conv10 = nn.Conv2d(64, out_ch, 1)

    def forward(self,x):
        # Down
        c1 = self.conv1(x)  # (N, 64, h, w)
        p1 = self.pool1(c1)  # (N, 64, h/2, w/2)

        c2 = self.conv2(p1)
        p2 = self.pool2(c2)  # (N, 128, h/4, w/4)

        c3 = self.conv3(p2)
        p3 = self.pool3(c3)  # (N, 256, h/8, w/8)

        c4 = self.conv4(p3)
        p4 = self.pool4(c4)  # (N, 512, h/16, w/16)

        c5 = self.conv5(p4)  # (N, 1024, h/16, w/16)
        
        # Up
        up_6 = self.up6(c5)  # (N, 512, h/8, w/8)
        merge6 = torch.cat([up_6, c4], dim=1)  # (N, 1024, h/8, w/8)
        c6 = self.conv6(merge6)  # (N, 512, h/8, w/8)

        up_7 = self.up7(c6)  # (N, 256, h/4, w/4)
        merge7 = torch.cat([up_7, c3], dim=1)  # (N, 512, h/4, w/4)
        c7 = self.conv7(merge7)  # (N, 256, h/4, w/4)

        up_8 = self.up8(c7)  # (N, 128, h/2, w/2)
        merge8 = torch.cat([up_8, c2], dim=1)  # (N, 256, h/2, w/2)
        c8 = self.conv8(merge8)  # (N, 128, h/2, w/2)

        up_9 = self.up9(c8)  # (N, 64, h, w)
        merge9 = torch.cat([up_9,c1],dim=1)  # (N, 128, h, w)
        c9 = self.conv9(merge9)  # (N, 64, h, w)

        c10 = self.conv10(c9)
        # out = nn.Sigmoid()(c10)

        return c10


In [25]:
x = torch.randn([2, 3, 256, 256])
model =  Unet_CBAM(3, 2)
out = model(x)
print(out.shape)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([2, 2, 256, 256])
